# Webinar n°2: Pyleecan Advanced - Part 2: SciDataTool

This notebook is the support of the second out of three webinars organized by the association [Green Forge Coop](https://www.linkedin.com/company/greenforgecoop/about/) 

The webinars schedule is:

Speakers: 

This webinar will be recorded and the video will be shared on [pyleecan.org](https://pyleecan.org/tutorials.html)

# Installation guidelines
To use this notebook please:
- Install Anaconda
- In Anaconda Prompt run the command "pip install pyleecan"
- Install the latest version of [femm](http://www.femm.info/wiki/Download) (windows only)
- In Anaconda Navigator, lanch Jupyter Notebook
- Jupyter Notebook should open a tab in your web brower, select this notebook to open it

To check if everything is correctly set, please run the following cell.

**WARNING**: this file is an archive, the webinar used pyleecan X.Y.Z and SciDataTool X.Y.Z

In [ ]:
%matplotlib notebook
from datetime import date
print("Running date:", date.today().strftime("%B %d, %Y"))
import pyleecan
print("Pyleecan version:" + pyleecan.__version__)
import SciDataTool
print("SciDataTool version:" + SciDataTool.__version__)

# Load the machine
from os.path import join
from pyleecan.Functions.load import load
from pyleecan.definitions import DATA_DIR

Toyota_Prius = load(join(DATA_DIR, "Machine", "Toyota_Prius.json"))
Toyota_Prius.plot()

# Check FEMM installation
from pyleecan.Classes._FEMMHandler import _FEMMHandler

femm = _FEMMHandler()
femm.openfemm(0)
femm.closefemm()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.art3d as art3d

## 1) Pyleecan simulation

To demonstrate the capabilities and the use of the SciDataTool objects, two simulations are launched with FEMM: with imposed currents and in open-circuit. The simulations use periodicity and parallelization to reduce execution time.

In [ ]:
from os import chdir
chdir('..')

# Import Pyleecan modules
from numpy import exp, sqrt, pi
from os.path import join
from pyleecan.Classes.Simu1 import Simu1
from pyleecan.Classes.InputCurrent import InputCurrent
from pyleecan.Classes.MagFEMM import MagFEMM
from pyleecan.Classes.ForceMT import ForceMT
from pyleecan.Classes.Output import Output
from pyleecan.Functions.load import load
from pyleecan.definitions import DATA_DIR

# Load the machine
Toyota_Prius = load(join(DATA_DIR, "Machine", "Toyota_Prius.json"))

# Simulation initialization
simu = Simu1(name="FEMM_periodicity", machine=Toyota_Prius)

# Definition of the enforced output of the electrical module
simu.input = InputCurrent(
    Na_tot=252 * 8,
    Nt_tot=50 * 8,
    N0=1000,
)
# Set Id/Iq according to I0/Phi0
simu.input.set_Id_Iq(I0=250 / sqrt(2), Phi0=140*pi/180)

# Definition of the magnetic simulation: with periodicity
simu.mag = MagFEMM(is_periodicity_a=True, is_periodicity_t=True, nb_worker=4)
simu.force = ForceMT(is_periodicity_a=True, is_periodicity_t=True)

# Definition of the open-circuit simulation
simu2 = simu.copy()
simu2.input.Id_ref = 0
simu2.input.Iq_ref = 0

# Run simulations
out = simu.run()
out2 = simu2.run()

## 2) How to extract data

Many output data are stored in **SciDataTool** formats. Here, we take for example the torque, for which the periodic part is stored along with its time axis.

To access data stored in **SciDataTool** objects, we use `get_along` methods as seen previously:

In [ ]:
print(out.mag.Tem.get_along("time"))

By default, the method reconstructs the whole periodic signal. To extract only the periodic part, which can be useful in computations, use `oneperiod`, `antiperiod` or `smallestperiod`:

In [ ]:
print(out.mag.Tem.get_along("time[smallestperiod]"))

## 3) How to plot data

**Pyleecan** has generic built-in plot commands, which allow to easily plot any field stored in a **SciDataTool** object:

- `plot_2D`, `plot_3D`, `plot_4D`: plot functions which take lists or arrays, labels, titles, etc, to unify the plot formatting
- `plot_2D_Data` and `plot_3D_Data`: plot methods (of the *Output* object) and functions which take **SciDataTool** objects to automatically build titles, labels, etc.

The plot commands use the same syntax as the `get_along` methods:

In [ ]:
%matplotlib notebook
out.mag.Tem.plot_2D_Data("time")

To plot only one period or anti-period:

In [ ]:
out.mag.Tem.plot_2D_Data("time[smallestperiod]")

**pyleecan** also has `VectorField` objects for fields with several components, such as the air-gap flux density **B**. The plot command allows to plot all components:

In [ ]:
out.mag.B.plot_2D_Data("time")

To plot only one or several specific components, one can use the `component_list` parameter:

In [ ]:
out.mag.B.plot_2D_Data("time", component_list=["radial"])

## 4) How to slice data

**SciDataTool** also allows to slice along a specific axis, and for specific indices or values. The same feature is also available in the plot commands.

For example, in the last plot, $B_r$ was sliced at `angle[0]` (slice by default), but we could want to slice at another angle:

In [ ]:
out.mag.B.plot_2D_Data("time", "angle=pi/6", component_list=["radial"])

We can even superimpose several slices:

In [ ]:
out.mag.B.plot_2D_Data("time", "angle[0,10,20]", component_list=["radial"])

## 5) How to compute and plot Fourier Transforms

One of the main interest of **SciDataTool** is to hide the complexity of Fourier Transforms. To demonstrate this feature, let us try to compute and plot the fft of the torque manually, and then using **SciDataTool** methods.

- manually:

In [ ]:
# Extract raw data
Tem_raw = out.mag.Tem.values
t0 = out.mag.Time.initial
tf = out.mag.Time.final
Nt_tot = out.mag.Time.number

# Reconstruct data
time_vector = np.linspace(t0, tf, Nt_tot, endpoint=False)
Tem_array = np.tile(Tem_raw, 8)

# Compute fft
Tem_FT = np.fft.fft(Tem_array)
Tem_FT[0] = Tem_FT[0] * 0.5
Tem_FT = 2 * np.fft.fftshift(Tem_FT) / Nt_tot
Tem_FT_mag = np.abs(Tem_FT)

# Compute freqs
timestep = float(time_vector[1] - time_vector[0])  # Sample step
fsampt = 1.0 / timestep  # Sample frequency
freqscale = Nt_tot / fsampt
freqs = [i - int(Nt_tot / 2) for i in range(int(Nt_tot))]
freqs = np.array([i / freqscale for i in freqs])

# Plot results
from SciDataTool.Functions.Plot.plot_2D import plot_2D
%matplotlib notebook
plot_2D([freqs], [Tem_FT_mag], type_plot="bargraph", x_min=-50, x_max=1000)

- using **SciDataTool** methods and **pyleecan** plot commands:

In [ ]:
# Equivalent process using SciDataTool
print(out.mag.Tem.get_magnitude_along("freqs=[0,1000]"))

# Plot using generic plot command
out.mag.Tem.plot_2D_Data("freqs=[0,1000]")

In the case of spatial Fourier Transform, the fundamental is automatically detected from the machine data:

In [ ]:
out.mag.B.plot_2D_Data("wavenumber=[0,100]", component_list=["radial"])

## 6) How to convert or normalize data

Another interesting feature of **SciDataTool** is the conversion feature: **SciDataTool** has built-in methods to convert units, and normalize fields and axes.

For example, we can plot the torque fft in electrical orders:

In [ ]:
out.mag.Tem.plot_2D_Data("freqs->elec_order=[0,10]")

In the following example, we will convert the stator winding flux from Webers into Maxwells ($1$ Wb = $10^8$ Mx):

In [ ]:
out.mag.Phi_wind_stator.plot_2D_Data("time", "phase")

In [ ]:
out.mag.Phi_wind_stator.plot_2D_Data("time", "phase", unit="Mx")

In this last example, we will convert the angle axis into degrees and for the air-gap flux density from Teslas into Gauss ($1$ T= $10^4$ G)

In [ ]:
out.mag.B.plot_2D_Data("angle", component_list=["radial"])

In [ ]:
out.mag.B.plot_2D_Data("angle{°}", component_list=["radial"], unit="G")

The angle can also be converted into distance:

In [ ]:
out.mag.B.plot_2D_Data("angle->distance", component_list=["radial"])

## 7) How to compare data

**SciDataTool** also allows to compare several fields, even if they are defined on different discretizations. To do so, **SciDataTool** has `compare_along` methods, and **pyleecan** plot commands have a `data_list` parameter.

In the folowing example, we compare the torque from the reference simulation, and from the open-circuit one:

In [ ]:
out.mag.B.plot_2D_Data(
    "time", component_list=["radial"], data_list=[out2.mag.B], legend_list=["Reference", "Open-circuit"]
)

We can also compare fft:

In [ ]:
out.mag.B.plot_2D_Data(
    "freqs=[0,1000]", component_list=["radial"], data_list=[out2.mag.B], legend_list=["Reference", "Open-circuit"]
)

## 8) 3D plots

3D plots (for data defined on at least two axes) can also be created using the `plot_3D_Data` command:

In [ ]:
out.mag.B.plot_3D_Data("time", "angle{°}", component_list=["radial"])

To visualize this surface from above, use `is_2D_view`:

In [ ]:
out.mag.B.plot_3D_Data("time", "angle{°}", component_list=["radial"], is_2D_view=True)

The field is automatically reconstructed using the periodicities specified in the simulation. It is always possible to plot a single period:

In [ ]:
out.mag.B.plot_3D_Data("time[smallestperiod]", "angle[smallestperiod]{°}", component_list=["radial"], is_2D_view=True)

2D Fourier Transforms also rely on `plot_3D_Data` command:

In [ ]:
out.mag.B.plot_3D_Data(
    "freqs->elec_order=[0,10]", "wavenumber->space_order", N_stem=50, component_list=["radial"]
)

It can also be viewed in 2D:

In [ ]:
out.mag.B.plot_3D_Data("freqs=[0,1000]", "wavenumber=[-50,50]", component_list=["radial"], is_2D_view=True)

All these plots were made using the magnetic flux density, but they are of course applicable to any 2D output data, like the force computed using the Maxwell Stress Tensor:

In [ ]:
out.force.AGSF.plot_3D_Data("freqs=[0,1000]", "wavenumber=[-50,50]", component_list=["radial"], is_2D_view=True)